In [2]:
# Install dependencies
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 949.8/949.8 kB 31.9 MB/s eta 0:00:00


In [3]:
!nvidia-smi

Sun Mar 23 08:57:30 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla P100-PCIE-16GB           Off |   00000000:00:04.0 Off |                    0 |
| N/A   33C    P0             25W /  250W |       0MiB /  16384MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [4]:
import os
import shutil
import pandas as pd
from sklearn.model_selection import train_test_split
from ultralytics import YOLO

# Define paths
TRAIN_IMG_DIR = "/kaggle/input/dlp-object-detection-week-10/final_dlp_data/final_dlp_data/train/images"
TRAIN_LABEL_DIR = "/kaggle/input/dlp-object-detection-week-10/final_dlp_data/final_dlp_data/train/labels"
TEST_IMG_DIR = "/kaggle/input/dlp-object-detection-week-10/final_dlp_data/final_dlp_data/test/images"
CLASS_NAMES = {0: "aegypti", 1: "albopictus", 2: "anopheles", 3: "culex", 4: "culiseta", 5: "japonicus/koreicus"}

# Step 1: Prepare dataset
WORKING_DIR = "/kaggle/working/mosquito_dataset"
os.makedirs(f"{WORKING_DIR}/train/images", exist_ok=True)
os.makedirs(f"{WORKING_DIR}/train/labels", exist_ok=True)
os.makedirs(f"{WORKING_DIR}/val/images", exist_ok=True)
os.makedirs(f"{WORKING_DIR}/val/labels", exist_ok=True)

# Get all image files
image_files = [f for f in os.listdir(TRAIN_IMG_DIR) if f.endswith('.jpeg')]
label_files = [f.replace('.jpeg', '.txt') for f in image_files]

# Split into train (80%) and val (20%)
train_imgs, val_imgs = train_test_split(image_files, test_size=0.2, random_state=42)

# Copy files
for img in train_imgs:
    shutil.copy(os.path.join(TRAIN_IMG_DIR, img), f"{WORKING_DIR}/train/images/{img}")
    shutil.copy(os.path.join(TRAIN_LABEL_DIR, img.replace('.jpeg', '.txt')), 
                f"{WORKING_DIR}/train/labels/{img.replace('.jpeg', '.txt')}")
for img in val_imgs:
    shutil.copy(os.path.join(TRAIN_IMG_DIR, img), f"{WORKING_DIR}/val/images/{img}")
    shutil.copy(os.path.join(TRAIN_LABEL_DIR, img.replace('.jpeg', '.txt')), 
                f"{WORKING_DIR}/val/labels/{img.replace('.jpeg', '.txt')}")

# Create data.yaml
yaml_content = f"""
train: {WORKING_DIR}/train/images
val: {WORKING_DIR}/val/images
test: {TEST_IMG_DIR}
nc: 6
names: {list(CLASS_NAMES.values())}
"""
with open(f"{WORKING_DIR}/data.yaml", 'w') as f:
    f.write(yaml_content)

print("Dataset prepared!")

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Dataset prepared!


In [5]:
# Step 2: Train YOLOv8
model = YOLO("yolov8n.pt")  # Nano model for speed

model.train(
    data=f"{WORKING_DIR}/data.yaml",
    epochs=50,           # 1.5 hours on Kaggle GPU P100
    imgsz=640,           # Standard input size
    batch=16,            
    device=0,            
    patience=10          # Early stopping
)

100%|██████████| 6.25M/6.25M [00:00<00:00, 183MB/s]


Ultralytics 8.3.94 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/kaggle/working/mosquito_dataset/data.yaml, epochs=50, time=None, patience=10, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, sh

100%|██████████| 755k/755k [00:00<00:00, 45.1MB/s]


Overriding model.yaml nc=80 with nc=6

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

100%|██████████| 5.35M/5.35M [00:00<00:00, 44.3MB/s]


...

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 47/47 [00:22<00:00,  2.09it/s]


                   all       1500       1500      0.732      0.496      0.457      0.351
               aegypti         10         10          1          0     0.0287     0.0175
            albopictus        677        677      0.821      0.942      0.923      0.703
             anopheles          9          9          1          0     0.0234     0.0154
                 culex        661        661      0.829      0.926      0.921      0.712
              culiseta         85         85      0.516      0.694      0.616       0.48
    japonicus/koreicus         58         58      0.227      0.414      0.229      0.179


/usr/local/lib/python3.10/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.10/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


Speed: 0.2ms preprocess, 1.5ms inference, 0.0ms loss, 1.6ms postprocess per image
Results saved to runs/detect/train


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4, 5])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7bd2eb10f8e0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
     

In [6]:
import os
import pandas as pd
from ultralytics import YOLO

# Paths
TEST_IMG_DIR = "/kaggle/input/dlp-object-detection-week-10/final_dlp_data/final_dlp_data/test/images"
CLASS_NAMES = {0: "aegypti", 1: "albopictus", 2: "anopheles", 3: "culex", 4: "culiseta", 5: "japonicus/koreicus"}

# Load trained model
model = YOLO("/kaggle/working/runs/detect/train/weights/best.pt")

# Run inference
results = model.predict(source=TEST_IMG_DIR, conf=0.25, iou=0.5)

# Format submission
submission_lines = []
test_images = sorted([f for f in os.listdir(TEST_IMG_DIR) if f.endswith('.jpeg')])  # 525 images
for img_idx, (img_name, result) in enumerate(zip(test_images, results)):
    preds = []
    if result.boxes:  # If detections exist
        img_height, img_width = result.orig_shape  # Original image dimensions
        for box in result.boxes:
            class_id = int(box.cls.item())
            conf = float(box.conf.item())
            x_center, y_center, width, height = box.xywh[0]  # Pixel coords
            # Normalize to 0-1
            x_center = float(x_center) / img_width
            y_center = float(y_center) / img_height
            width = float(width) / img_width
            height = float(height) / img_height
            preds.append([class_id, conf, x_center, y_center, width, height])
    # Select highest-confidence prediction
    if preds:
        best_pred = max(preds, key=lambda x: x[1])  # Max by conf (index 1)
        class_id, conf, x_center, y_center, width, height = best_pred
        submission_lines.append([img_idx, img_name, CLASS_NAMES[class_id], conf, x_center, y_center, width, height])
    else:
        submission_lines.append([img_idx, img_name, CLASS_NAMES[0], 0.0, 0.5, 0.5, 0.1, 0.1])

# Create submission.csv
submission_df = pd.DataFrame(submission_lines, columns=["id", "ImageID", "LabelName", "Conf", 
                                                        "xcenter", "ycenter", "bbx_width", "bbx_height"])
submission_df.to_csv("/kaggle/working/submission.csv", index=False)
print("Submission rows:", len(submission_df))
print(submission_df.head(10))


image 1/525 /kaggle/input/dlp-object-detection-week-10/final_dlp_data/final_dlp_data/test/images/0031063e-716a-4080-934c-77598dc8de72.jpeg: 640x480 1 culex, 36.5ms
image 2/525 /kaggle/input/dlp-object-detection-week-10/final_dlp_data/final_dlp_data/test/images/00fbfad7-9722-4581-831c-79faa576ea7f.jpeg: 640x480 (no detections), 6.7ms
image 3/525 /kaggle/input/dlp-object-detection-week-10/final_dlp_data/final_dlp_data/test/images/02043b0e-3d7d-4ca4-a36f-4bf97c344264.jpeg: 640x384 1 culex, 37.1ms
image 4/525 /kaggle/input/dlp-object-detection-week-10/final_dlp_data/final_dlp_data/test/images/0365513c-8f00-44f3-abd0-1fadde81c602.jpeg: 640x480 1 culex, 7.4ms
image 5/525 /kaggle/input/dlp-object-detection-week-10/final_dlp_data/final_dlp_data/test/images/0365d78b-2064-4d54-b15c-994d1950479a.jpeg: 640x512 1 albopictus, 36.5ms
image 6/525 /kaggle/input/dlp-object-detection-week-10/final_dlp_data/final_dlp_data/test/images/03d6749a-c5b3-45a3-81fa-b9c2f09eb4ba.jpeg: 576x640 1 albopictus, 35.8ms


image 1/525 /kaggle/input/dlp-object-detection-week-10/final_dlp_data/final_dlp_data/test/images/0031063e-716a-4080-934c-77598dc8de72.jpeg: 640x480 1 culex, 7.5ms
image 2/525 /kaggle/input/dlp-object-detection-week-10/final_dlp_data/final_dlp_data/test/images/00fbfad7-9722-4581-831c-79faa576ea7f.jpeg: 640x480 (no detections), 6.8ms
image 3/525 /kaggle/input/dlp-object-detection-week-10/final_dlp_data/final_dlp_data/test/images/02043b0e-3d7d-4ca4-a36f-4bf97c344264.jpeg: 640x384 1 culex, 8.3ms
image 4/525 /kaggle/input/dlp-object-detection-week-10/final_dlp_data/final_dlp_data/test/images/0365513c-8f00-44f3-abd0-1fadde81c602.jpeg: 640x480 1 culex, 6.8ms
image 5/525 /kaggle/input/dlp-object-detection-week-10/final_dlp_data/final_dlp_data/test/images/0365d78b-2064-4d54-b15c-994d1950479a.jpeg: 640x512 1 albopictus, 7.3ms
image 6/525 /kaggle/input/dlp-object-detection-week-10/final_dlp_data/final_dlp_data/test/images/03d6749a-c5b3-45a3-81fa-b9c2f09eb4ba.jpeg: 576x640 1 albopictus, 6.9ms
ima

In [7]:
submission_df = submission_df.rename(columns={"Id": "id"})
submission_df.to_csv("/kaggle/working/submission.csv", index=False)
print("Submission rows:", len(submission_df))
print(submission_df.head(10))

Submission rows: 525
   id                                    ImageID   LabelName      Conf  \
0   0  0031063e-716a-4080-934c-77598dc8de72.jpeg       culex  0.553926   
1   1  00fbfad7-9722-4581-831c-79faa576ea7f.jpeg     aegypti  0.000000   
2   2  02043b0e-3d7d-4ca4-a36f-4bf97c344264.jpeg       culex  0.933766   
3   3  0365513c-8f00-44f3-abd0-1fadde81c602.jpeg       culex  0.880821   
4   4  0365d78b-2064-4d54-b15c-994d1950479a.jpeg  albopictus  0.387837   
5   5  03d6749a-c5b3-45a3-81fa-b9c2f09eb4ba.jpeg  albopictus  0.537925   
6   6  03ef458e-8f40-4d42-811b-f3c2db667e12.jpeg  albopictus  0.758071   
7   7  04741ad4-6e1c-416d-acd4-b352cf18ebda.jpeg     aegypti  0.000000   
8   8  04a95170-0348-43e1-a722-e178370e6812.jpeg     aegypti  0.000000   
9   9  04d4d98b-4b4d-45d1-924a-82f4c3a9ff27.jpeg       culex  0.894057   

    xcenter   ycenter  bbx_width  bbx_height  
0  0.494537  0.559763   0.148524    0.125082  
1  0.500000  0.500000   0.100000    0.100000  
2  0.594113  0.466225  

In [8]:
import pandas as pd
submission = pd.read_csv("/kaggle/working/submission.csv")
print(submission.head(10))
print("Unique LabelName:", submission["LabelName"].unique())
print("Conf range:", submission["Conf"].min(), "to", submission["Conf"].max())
print("Coord ranges:", [submission[col].min() for col in ["xcenter", "ycenter", "bbx_width", "bbx_height"]], 
      "to", [submission[col].max() for col in ["xcenter", "ycenter", "bbx_width", "bbx_height"]])

   id                                    ImageID   LabelName      Conf  \
0   0  0031063e-716a-4080-934c-77598dc8de72.jpeg       culex  0.553926   
1   1  00fbfad7-9722-4581-831c-79faa576ea7f.jpeg     aegypti  0.000000   
2   2  02043b0e-3d7d-4ca4-a36f-4bf97c344264.jpeg       culex  0.933766   
3   3  0365513c-8f00-44f3-abd0-1fadde81c602.jpeg       culex  0.880821   
4   4  0365d78b-2064-4d54-b15c-994d1950479a.jpeg  albopictus  0.387837   
5   5  03d6749a-c5b3-45a3-81fa-b9c2f09eb4ba.jpeg  albopictus  0.537925   
6   6  03ef458e-8f40-4d42-811b-f3c2db667e12.jpeg  albopictus  0.758071   
7   7  04741ad4-6e1c-416d-acd4-b352cf18ebda.jpeg     aegypti  0.000000   
8   8  04a95170-0348-43e1-a722-e178370e6812.jpeg     aegypti  0.000000   
9   9  04d4d98b-4b4d-45d1-924a-82f4c3a9ff27.jpeg       culex  0.894057   

    xcenter   ycenter  bbx_width  bbx_height  
0  0.494537  0.559763   0.148524    0.125082  
1  0.500000  0.500000   0.100000    0.100000  
2  0.594113  0.466225   0.500515    0.432515

In [9]:
import os
model_path = "/kaggle/working/runs/detect/train/weights/best.pt"
if os.path.exists(model_path):
    print(f"Model found at {model_path}, size: {os.path.getsize(model_path) / 1024**2:.2f} MB")
else:
    print("Model not found!")

Model found at /kaggle/working/runs/detect/train/weights/best.pt, size: 5.93 MB


In [10]:
import shutil
shutil.make_archive("/kaggle/working/trained_model", "zip", "/kaggle/working/runs/detect/train")
print("Zipped model saved at /kaggle/working/trained_model.zip")

Zipped model saved at /kaggle/working/trained_model.zip


In [11]:
import pandas as pd

test_csv_path = "/kaggle/working/submission.csv"  # The CSV to reorder
submission_csv_path = "/kaggle/input/dlp-object-detection-week-10/sample_submission.csv"  # The CSV with correct Image_ID order

# Load both CSV files
test_df = pd.read_csv(test_csv_path)
submission_df = pd.read_csv(submission_csv_path)

# Merge based on 'ImageID' to reorder test_df
reordered_test_df = submission_df[['ImageID']].merge(test_df, on='ImageID', how='left')

# Reorder columns to have 'id' as the first column
column_order = ['id', 'ImageID', 'LabelName', 'Conf', 'xcenter', 'ycenter', 'bbx_width', 'bbx_height']
reordered_test_df = reordered_test_df[column_order]

# Reset the 'id' column to start from 0
reordered_test_df['id'] = range(len(reordered_test_df))


test_csv_path1 = "/kaggle/working/final_submission.csv"
# Save the reordered file back to 21f3001013.csv
reordered_test_df.to_csv(test_csv_path1, index=False)

print(f"file saved in {test_csv_path1}")
print(reordered_test_df.head())

file saved in /kaggle/working/final_submission.csv
   id                                    ImageID   LabelName      Conf  \
0   0  16dbbc39-fcb6-498e-963f-d3807fcc3cec.jpeg  albopictus  0.849844   
1   1  7daa68bd-928b-4b43-850f-4626f16d2c14.jpeg  albopictus  0.931435   
2   2  bb0986d9-fa04-40a5-8de2-c54045537e17.jpeg  albopictus  0.649377   
3   3  11924d04-69fe-469d-8d1c-119bc0f0b94f.jpeg    culiseta  0.532639   
4   4  db4ade79-fce4-4d77-a8fa-36b9e77dfd08.jpeg       culex  0.431713   

    xcenter   ycenter  bbx_width  bbx_height  
0  0.432593  0.450257   0.136080    0.113586  
1  0.499103  0.559967   0.448333    0.300856  
2  0.359852  0.467254   0.114500    0.168810  
3  0.622626  0.628378   0.754748    0.743244  
4  0.367333  0.469168   0.186540    0.128788  
